In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

# 4.4.2 NLP As A Supervised Problem

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)

# Take subset of data due to memory limits
alice = text_cleaner(alice[0:35000])
persuasion = text_cleaner(persuasion[0:35000])

In [3]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [6]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [7]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [8]:
# Create data frame with features. This can take a while
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500


,allowance,fancy,recommend,sight,that,early,walrus,feature,forbad,fender,...,rugged,urge,prematurely,shrubbery,where,mean,occupy,disgust,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW
Now let's give the bag of words features a whirl by trying a random forest.


In [9]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)

# Take 1700 words.  This will come into play further below when we compare to Emma.
X = X.iloc[:, :1700]
X.head()

,allowance,fancy,recommend,sight,that,early,walrus,feature,forbad,fender,...,sentiment,naval,fright,dry,bank,scold,mother,some,learn,reply
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 1.0

Test set score: 0.8552631578947368


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [11]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(340, 1700) (340,)
Training set score: 0.9941176470588236

Test set score: 0.8728070175438597


Logistic regression performs a bit better than the random forest.  

## BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [12]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.9911764705882353

Test set score: 0.8421052631578947


Looks like logistic regression is the winner, but there's room for improvement.

## Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [13]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [14]:
# Parse our cleaned data.
emma_doc = nlp(emma[0:40000])

In [15]:
# Group into sentences.
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

In [16]:
emmawords = bag_of_words(emma_doc)
common_words_2 = set(emmawords + alicewords)
sentences_2 = pd.DataFrame(emma_sents + alice_sents)

word_counts_2 = bow_features(sentences_2, common_words_2)

Processing row 0
Processing row 500


In [17]:
Y_2 = word_counts_2['text_source']
X_2 = word_counts_2.drop(['text_sentence', 'text_source'], 1)
X_2 = X_2.iloc[:, :1700]

In [18]:
# Now we can model it!
# Let's use logistic regression again.

# Create new test set with Alice and Emma
X_train2, X_test2, y_train2, y_test2 = train_test_split(
    X_2, 
    Y_2, 
    test_size=0.4,
    random_state=42
)

print(X_test2.shape)
print(y_test2.shape)

(268, 1700)
(268,)


In [19]:
# Model.
print('\nTest set score:', lr.score(X_test2, y_test2))
lr_2_predicted = lr.predict(X_test2)
pd.crosstab(y_test2, lr_2_predicted)


Test set score: 0.5261194029850746


col_0,Austen,Carroll
text_source,,
Austen,23,113
Carroll,14,118


Results are not so great but probably more a function of the limited text size due to memory limits.

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93% (87% with limited data here).  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

* Why are we supposed to design models on the test set?  Doesn't make sense...don't you design on train set?

### Try with SVM

In [20]:
from sklearn.svm import SVC

svc = SVC()
train = svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))

(340, 1700) (340,)
Training set score: 0.6411764705882353

Test set score: 0.6403508771929824


SVM does not perform well at all, although it doesn't overfit like the other models.

### Add punctuation to features
Adds 13 columns

In [21]:
# Utility function to create a list of the 2000 most common words.
def words_punct(text):
    
    # Filter out stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_stop
                ]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords_punct = words_punct(alice_doc)
persuasionwords_punct = words_punct(persuasion_doc)

# Combine bags to create a set of unique words.
common_words_punct = set(alicewords_punct + persuasionwords_punct)

# Create data frame with features. This can take a while
word_counts_punct = bow_features(sentences, common_words_punct)
word_counts_punct.head()

Processing row 0
Processing row 500


,allowance,fancy,recommend,sight,that,early,walrus,feature,forbad,fender,...,rugged,urge,prematurely,shrubbery,where,mean,occupy,disgust,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [22]:
# Test new features using best model from above (logistic)

Y = word_counts_punct['text_source']
X = word_counts_punct.drop(['text_sentence', 'text_source'], 1)

# Split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

lr = LogisticRegression()
lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(340, 1775) (340,)
Training set score: 0.9911764705882353

Test set score: 0.9342105263157895


Adding punctuation increased accuracy on the test set by 6% (87% -> 93%) [Alice vs. Persuasion]



# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [23]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

### Get new book (Leaves by Whitman).  Clean and process into sentences.

In [27]:
# Grab Whitman, clean and process
whitman = gutenberg.raw('whitman-leaves.txt')
whitman = re.sub(r'VOLUME \w+', '', whitman)
whitman = re.sub(r'CHAPTER \w+', '', whitman)
whitman = text_cleaner(whitman)
print(whitman[:100])

Come, said my soul, Such verses for my Body let us write, (for we are one,) That should I after retu


In [28]:
# Parse into doc using SpaCy
whitman_doc = nlp(whitman[0:25000])

### Setup and run model: Alice vs. Whitman

Model appears equally effective at identifying Alice vs. Whitman Leaves.

In [29]:
# Bag words
whitmanwords_punct = words_punct(whitman_doc)
alicewords_punct = words_punct(alice_doc)

# Combine Whitman and Alice bags
common_words = set(whitmanwords_punct + alicewords_punct)

# Group into sentences
whitman_sents = [[sent, 'Whitman'] for sent in whitman_doc.sents]

# Combine Whitman and Alice sentences
sentences = pd.DataFrame(whitman_sents + alice_sents)

# Get BoW Features
word_counts = bow_features(sentences, common_words)

Processing row 0
Processing row 500


In [30]:
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)

In [31]:
from sklearn.model_selection import cross_val_score
print('Scores:', cross_val_score(lr, X, Y, cv=5))
print('Avg:', np.mean(cross_val_score(lr, X, Y, cv=5)))

Scores: [0.92307692 0.94017094 0.90598291 0.87068966 0.93043478]
Avg: 0.9140710414023758


### Setup and run model: Persuasion vs Whitman

The model is still good at identifying Austen's Persuasion vs. Whitman's Leaves, although there is quite a bit more variation between folds compared to Whitman vs. Alice.

In [32]:
# Combine Whitman and Persuasion bags
common_words = set(whitmanwords_punct + persuasionwords_punct)

# Combine Whitman and Persuasion sentences
sentences = pd.DataFrame(whitman_sents + persuasion_sents)

# Get BoW Features
word_counts = bow_features(sentences, common_words)

Processing row 0


In [33]:
Y = word_counts['text_source']
X = word_counts.drop(['text_sentence', 'text_source'], 1)

In [34]:
from sklearn.model_selection import cross_val_score
print('Scores:', cross_val_score(lr, X, Y, cv=5))
print('Avg:', np.mean(cross_val_score(lr, X, Y, cv=5)))

Scores: [0.96470588 0.95294118 0.88235294 0.91666667 0.85542169]
Avg: 0.9144176706827309
